<a href="https://colab.research.google.com/github/mayank-cse/DEV---A-Face-Identification-Software/blob/main/testVideo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install facenet-pytorch
!pip install -v -e
# !pip install mmcv-full
!pip install -U openmim
# !mim install mmcv-full
!pip install mmcv
!pip install scikit-image
!pip install -q streamlit
!pip install pyngrok

In [ ]:
!pip install -q streamlit
!pip install pyngrok

In [ ]:
# %cd .. 
%cd /content/gdrive/MyDrive/FaceRecog
# !git clone https://github.com/timesler/facenet-pytorch.git

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

FACE DETECTION

In [ ]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)

In [ ]:
video = mmcv.VideoReader('/content/maskPerson.jpeg')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

display.Video('/content/gdrive/MyDrive/FaceRecog/face-recognition/Test/TestVideoShort.mp4', width=640)

In [ ]:
# !pip install -U kora
# from kora.drive import upload_public
# url = upload_public('/content/gdrive/MyDrive/FaceRecog/face-recognition/Test/TestVideoShort.mp4')
# # then display it
# from IPython.display import HTML
# HTML(f"""<video src={url} width=500 controls/>""")

In [ ]:
frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    print(boxes)
    if boxes is None:
      continue
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')

In [ ]:
d = display.display(frames_tracked[0], display_id=True)
i = 1
try:
    while True:
        d.update(frames_tracked[i % len(frames_tracked)])
        i += 1
except KeyboardInterrupt:
    pass

In [ ]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('/content/maskPerson.jpeg', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()

Data Augmentation

In [ ]:
import cv2
import albumentations as A
import numpy as np
# from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from PIL import Image
from matplotlib import *
from matplotlib.pyplot import *


image = cv2.imread("/content/5.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
bboxes = [[13, 170, 224, 410]]

# Pascal_voc (x_min, y_min, x_max, y_max), YOLO, COCO

transform = A.Compose(
    [
        A.Resize(width=1920, height=1080),
        A.RandomCrop(width=1280, height=720),
        A.Rotate(limit=40, p=0.9, border_mode=cv2.BORDER_CONSTANT),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.9),
        A.OneOf([
            A.Blur(blur_limit=3, p=0.5),
            A.ColorJitter(p=0.5),
        ], p=1.0),
    ], bbox_params=A.BboxParams(format="pascal_voc", min_area=2048,
                                min_visibility=0.3, label_fields=[])
)

images_list = [image]
saved_bboxes = [bboxes[0]]
for i in range(15):
    augmentations = transform(image=image, bboxes=bboxes)
    augmented_img = augmentations["image"]

    if len(augmentations["bboxes"]) == 0:
        continue
    visualize(augmented_img)
    images_list.append(augmented_img)
    saved_bboxes.append(augmentations["bboxes"][0])

AUGMENTATION

In [ ]:
!pip install keras

import os

data_dir = '/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/test_images' 
os.listdir(data_dir)  
from keras.preprocessing.image import ImageDataGenerator
   
datagen = ImageDataGenerator(
        rotation_range = 40,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        brightness_range = (0.5, 1.5),
        featurewise_center=True,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=True,
        width_shift_range=0.2,
        height_shift_range=0.1,
        channel_shift_range=0.1,
        fill_mode='nearest',
        cval=0.,
        rescale=None,
        preprocessing_function=None)

for image_class in os.listdir(data_dir): 

        i = 0

        dir = "/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/test_images/"+image_class
        aug = "/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/test_images_augmented/"+image_class

        for batch in datagen.flow_from_directory(directory=dir, batch_size = 8,
                          save_to_dir =aug, 
                          save_prefix ='aug', save_format ='jpg'):
                i += 1
                
                if i > 100:
                        break

FACE RECOGNITION

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

# importing libraries
from PIL import Image
import cv2
import time
import os

workers = 0 if os.name == 'nt' else 4

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
def collate_fn(x):
    return x[0]
# Reading data from the folder
dataset = datasets.ImageFolder('/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/test_images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()} #Accessing names of people from folder names
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [ ]:
aligned = []
names = []
#x is image and y is index
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

In [ ]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [ ]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

# save data
data = [aligned, names] 
torch.save(data, '/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/data.pt') # saving data.pt file

In [ ]:
from google.colab.patches import cv2_imshow
# Using webcam recognize face

# loading data.pt file
load_data = torch.load('/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

cam = cv2.VideoCapture('/content/mgtest.png') 

while True:
    ret, frame = cam.read()
    if not ret:
        print("fail to grab frame, try again")
        break
        
    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    print(mtcnn(img, return_prob=True))
    if img_cropped_list is not None:
        # print(1)
        boxes, _ = mtcnn.detect(img)
        print(boxes)
        
        # prob_list = prob_list.tolist()
        
        # print(enumerate(prob_list))
        # for i, prob in enumerate(prob_list):
        print((prob_list))
        # prob_list = [[i] for i in prob_list]
        # prob_list = list(prob_list)
        for i, prob in enumerate(img_cropped_list):
            # print(1)
            # prob = prob_list[i]
            print(prob)
            prob = prob_list
            if prob>0.90:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                
                if min_dist<0.90:
                    frame = cv2.putText(frame, name+' '+str(min_dist), (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                
                frame = cv2.rectangle(frame, (box[0],box[1]) , (box[2],box[3]), (255,0,0), 2)

    cv2_imshow(frame)
        
    
    k = cv2.waitKey(1)
    if k%256==27: # ESC
        print('Esc pressed, closing...')
        break
        
    elif k%256==32: # space to save image
        print('Enter your name :')
        name = input()
        
        # create directory if not exists
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)

            
        img_name = "photos/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))
        
        
cam.release()
cv2.destroyAllWindows()
    

TUNING THE MODEL

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [ ]:
data_dir = '/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/test_images'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir+ '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()

PYTORCH YT Video

In [ ]:
# importing libraries

from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.io
import io

# from scipy.misc import impip save
from imageio import imwrite
from google.colab.patches import cv2_imshow

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float
from skimage import exposure
from PIL import Image, ImageEnhance



In [ ]:

# initializing MTCNN and InceptionResnetV1 

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
# mtcnn = MTCNN(keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval() 


MULTIPLE TEST IMAGE GENERATOR

In [ ]:
# Read data from folder

dataset = datasets.ImageFolder('/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/data/test_images_augmented') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True) 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0))
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx])        

# dists = [[(e1 - e2).norm().item() for e2 in embedding_list] for e1 in embedding_list]
# print(pd.DataFrame(dists, columns=name_list, index=name_list))

# save data
data = [embedding_list, name_list] 
torch.save(data, 'data.pt') # saving data.pt file

In [ ]:
from skimage.util import img_as_bool
from google.colab.patches import cv2_imshow
from skimage import exposure
# Using webcam recognize face


# COLOR BALANCER
def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100
    half_percent = percent / 200.0
    channels = cv2.split(img)

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)

        assert len(flat.shape) == 1
        flat = np.sort(flat)
        n_cols = flat.shape[0]

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img
def contrastInput(img):
    cv2_imshow(img)
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    # imwrite('C:/Users/mayan/OneDrive/Desktop/Human-Face-Detection-in-Excessive-Dark-Image-master/CodePython/face/Contrast stretching/test1.jpg',img_rescale)
    cv2_imshow(img_rescale)
    #gray-scale image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2_imshow(gray)
    #Applying clahe (Contrast Limited Adaptive Histogram Equalization)
    
    # clahe = cv2.createCLAHE(clipLimit = 5)
    clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
    final_img = clahe.apply(gray) + 30
    cv2_imshow(final_img)
    #Enhanced colored Image
    l_channel, a, b = cv2.split(img)
    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((final_img,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    cv2_imshow(enhanced_img)
    #Equalization
    equalized = cv2.equalizeHist(gray)
    cv2_imshow(equalized)
    # Equalization
    img_eq = exposure.equalize_hist(gray)
    # imwrite('C:/Users/mayan/OneDrive/Desktop/Human-Face-Detection-in-Excessive-Dark-Image-master/CodePython/face/Histogram equalization/test2.jpg',img_eq)
    cv2_imshow(img_eq)
    # Adaptive Equalization
    img_adapteq = exposure.equalize_adapthist(gray, clip_limit=0.03)
    # imwrite('C:/Users/mayan/OneDrive/Desktop/Human-Face-Detection-in-Excessive-Dark-Image-master/CodePython/face/Adaptive equalization/test3.jpg',img_adapteq)
    cv2_imshow(img_adapteq)

    #Brightness
    imgBright = increase_brightness(img, value=20)
    cv2_imshow(imgBright)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    h, s, v = cv2.split(hsv)
    v += 255
    final_hsv = cv2.merge((h, s, v))
    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    cv2_imshow(image)



    
# loading data.pt file
load_data = torch.load('/content/data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

cam = cv2.VideoCapture('/content/3.mp4') 
while True:
    ret, frame = cam.read()
    print(ret)
    # if frame is not None:
    #   frame = cv2.resize(frame, (700, 500))

    if not ret:
        print("fail to grab frame, try again")
        break
    
    # #Enhance Image Pixels
    # sr = cv2.dnn_superres.DnnSuperResImpl_create()
    # path = "/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/resolutionEnhancerModel/EDSR_x4.pb"
    # sr.readModel(path)
    # sr.setModel("edsr",4)
    # result = sr.upsample(frame)
    # # Resized image
    # resized = cv2.resize(result,dsize=None,fx=4,fy=4)
    # cv2_imshow(resized)
    # dst = cv2.detailEnhance(frame, sigma_s=10, sigma_r=0.15)
    # cv2_imshow(dst)
    # contrastInput(dst)
    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    print(prob_list)
    boxes, _ = mtcnn.detect(img)
    print(boxes) 
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
        print(prob_list)        
        for i, prob in enumerate(prob_list):
            if prob>0.80:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                
                if min_dist<0.85:
                    print(box[0],box[1])
                    frame = cv2.putText(frame, name+' '+str(min_dist), (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA, False)
                    print((box[0],box[1]) , (box[2],box[3]))
                frame = cv2.rectangle(frame, (int(box[0]),int(box[1])) , (int(box[2]),int(box[3])), (255,0,0), 2)
                
    
    cv2_imshow(frame)
    # cv2_imshow(frame)
        
    
    k = cv2.waitKey(1)
    if k%256==27: # ESC
        print('Esc pressed, closing...')
        break
        
    elif k%256==32: # space to save image
        print('Enter your name :')
        name = input()
        
        # create directory if not exists
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)
            
        img_name = "photos/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))
        
        
cam.release()
cv2.destroyAllWindows()
    

Hosting the application

In [ ]:
%%writefile webapp.py

import streamlit as st
import cv2
from PIL import Image
import numpy as np


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

rec=cv2.face.LBPHFaceRecognizer_create()
rec.read("trainingData.yml")
def detect_faces(our_image):
    img = np.array(our_image.convert('RGB'))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    # Draw rectangle around the faces
    name='Unknown'
    for (x, y, w, h) in faces:
        # To draw a rectangle in a face
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
        id, uncertainty = rec.predict(gray[y:y + h, x:x + w])
        print(id, uncertainty)

        if (uncertainty< 53):
            if (id == 1 or id == 3 or id == 5):
                name = "Nachiketa"
                cv2.putText(img, name, (x, y + h), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (0, 0, 255))
        else:
            cv2.putText(img, 'Unknown', (x, y + h), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (0, 0, 255))


    return img
def main():
    """Face Recognition App"""

    st.title("Streamlit Tutorial")

    html_temp = """
    <body style="background-color:red;">
    <div style="background-color:teal ;padding:10px">
    <h2 style="color:white;text-align:center;">Face Recognition WebApp</h2>
    </div>
    </body>
    """
    st.markdown(html_temp, unsafe_allow_html=True)

    image_file = st.file_uploader("Upload Image", type=['jpg', 'png', 'jpeg'])
    if image_file is not None:
        our_image = Image.open(image_file)
        st.text("Original Image")
        st.image(our_image)

    if st.button("Recognise"):
        result_img= detect_faces(our_image)
        st.image(result_img)


if __name__ == '__main__':
    main()

In [ ]:
! pip install streamlit -q
! pip install pyngrok
! pip install --upgrade protobuf


In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2JOk1LlPWzFvC9AWEkfebDsaorh_7uqdTpaKTJCUMQju78Muc") #ngrok.com

In [ ]:
# correct
!streamlit run webapp.py --server.port 80 &
url = ngrok.connect(port='80')
print(url)
    